In [2]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append('../')
from benchmarks.inference_models import *

models = (
        Banana(),
        Gaussian(ndims=100, eigenvalues='Gamma', numpy_seed= rng_inference_gym_icg),
        GermanCredit(),
        Brownian(),
        ItemResponseTheory(),
        StochasticVolatility()
)

dfs = [pd.read_csv('sequential_results/' + model.name + str(model.ndims) + '0.csv') for model in models]

round = lambda x: np.round(x, 3)
#round = lambda x: x

def get_mclmc(df, num_windows):
    return round(df[(df['sampler'] == 'mclmc:st3') & (df['num_windows'] == num_windows)]['ess_avg'].item())

def get_nuts(df):
    return round(df[df['sampler'] == 'nuts']['ess_avg'].item())

df = pd.DataFrame(np.array([[get_mclmc(df, 1) for df in dfs], [get_mclmc(df, 2) for df in dfs], [get_nuts(df) for df in dfs]]).T, columns= ['MCLMC1', 'MCLMC2', 'NUTS'])
df.insert(0, 'model', ['Banana', 'Ill Conditioned Gaussian', 'Sparse Logistic Regression', 'Brownian Motion', 'Item Response Theory', 'Stochastic Volatility'])

display(df)
df_old = pd.DataFrame([['Sparse Logistic Regression', 0.006, 0.0014], ['Stochastic Volatility', 0.011, 0.006]], columns = ['model', 'MCLMC1', 'NUTS'])
display(df_old)
#df = df.style.format(index=False, precision=3)
#print(df.to_latex(index= False))



,model,MCLMC1,MCLMC2,NUTS
0,Banana,0.019,0.020,0.002
1,Ill Conditioned Gaussian,0.003,0.003,0.001
2,Sparse Logistic Regression,0.024,0.024,0.010
3,Brownian Motion,0.049,0.059,0.019
4,Item Response Theory,0.063,0.062,0.014
5,Stochastic Volatility,0.000,0.000,0.003


,model,MCLMC1,NUTS
0,Sparse Logistic Regression,0.006,0.0014
1,Stochastic Volatility,0.011,0.0060


Take together different results

grads to low average bias:

#sequential NUTS  #sequential MCLMC    #ensemble MCLMC (total)   #ensemble MCLMC(per chain)    #large ensemble MLCMC (per chain)
      60000             33000                  50000                        500                             460
      2000              1500                   10000                        120                             100